In [3]:
%matplotlib qt5

from fmcal import fm_cal
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
cal = fm_cal()
cal.create_ini_file('tmp.ini')

To print the contents on the tmp.ini file into a new cell open a new cell with command "%load tmp.ini" and the contents of the file will be loaded into the cell as below

In [ ]:
# %load tmp.ini
[mcdmin]
# one or more pairs of id = min. roughness value.
0 = 0.004

[mcdmax]
# one or more pairs of id = max. roughness value.
0 = 0.010

[mcdinc]
# one or more pairs of id = increment values.
0 = 0.00025

[Params]
# relative or complete path\file.csv to measured water-surface elevation file.
meas_wse_file = ..\test\GR_wse.csv
# discharge value
q = 241.0
# downstream stage value
h_ds = 447.1
# set initial water-surface elevation boundary condition
# initype == 1 use upstream elevation (h_us)
# initype == 2 use step-backwater (onedcd = 1-d cd for step-backwater)
initype = 2
h_us = 449
onedcd = .015
# complete path to fastmech solver
solver_path = ;C:\Users\rmcd\iRICt\solvers\fastmech
# relative (to python code) or complete path to working directory where files/results are stored
working_dir = ..\test\cal_const_cd
# relative (to working directory) or complete path to base iric project (folder)
base_file = ..\test_const_cd\Case1.cgn
# file used to output results - save to working directory
rmse_file = test_rmse.csv



Example 1: Calibrating to a constant Cd.  The iRIC FaSTMECH (FM) project is located in the folder test/test_const_cd.  Note that the Geographic Data | roughness has 1 converage polygon encapsulated the entire grid and it's value is 0.  Make sure that these values are mapped to the grid (ie. From the menu Grid->Attributes Mapping->Execute.  Also that the Calculations Conditions have been specified and saved.  The following example illustrates how to vary the value of roughness over a range specified in the .ini file and determine a calibrated roughness for the reach. 

Note: For your own project you should spend some time calibrating the project in the iRIC application before launching this code to know that you have a good set of calculation conditions, boundary conditions, and some bounds as to what the roughness is.  

In [2]:
# magic command to capture output of model output in calibration loop.  
#  Output is displayed in next cell command
# comment line below if you want to see in real-time.  
#%%capture cal_output

cal = fm_cal('../test/test_constcd_config.ini')
cal.initialize()
#cal.create_ini_file()

fig = plt.figure()
ax = fig.add_subplot(111)
# cax = make_axes_locatable(ax).append_axes('right', size='5%', pad='2%')
plt.ion()

fig.show()
fig.canvas.draw()
x = np.arange(cal.mcdmin['0'], cal.mcdmax['0'], cal.mcdinc['0'])
numx = len(x)

tcd ={}
tcdind = {}
tcount = 0

for cdind0, cd0 in enumerate(x):
    tcd[0] = cd0
    tcdind[0] = cdind0
    simres = cal.update_var(tcount, tcd)
    # if tcount > 0:
    ax.plot(simres.loc[:cdind0, 'cd0'], simres.loc[:cdind0, 'rmse'], '-o')  # fit the line

    ax.set_xlabel('cd0')
    ax.set_ylabel('RMSE')

    fig.canvas.update()
    fig.canvas.flush_events()
    plt.pause(1)
    plt.draw()
    tcount+=1
#     fig.canvas.draw()   # draw
#     time.sleep(0.5)    #sleep
plt.savefig('testfig.png', dpi=300)
cal.resdf.to_csv(cal.rmse_file)
tmp = 0

In [6]:
#Display the captured simulated output
cal_output.show()

**********************************************************************
                                                                     *
                                                                     *
                          FASTMECH                                   *
 Flow And Sediment Transport and Morphological Evolution of CHannels *
 *    *   *        *             *            *             **       *
                                                                     *
   Developed by National Research Program, U.S. Geological Survey    *
                                                                     *
**********************************************************************
        327           0          39
WSE  0.000000000000000E+000  0.000000000000000E+000
Delete Clusters
          6
Cluster            1  Val         4885
Cluster            2  Val            1
Cluster            3  Val            1
Cluster            4  Val            1
Cluster           

The resulting testfig.png

![title](../test/cal_const_cd/testfig.png)

Alternatively plot as an interactive plot using the results stored in a the output file.  Hoever over points to see values.

In [4]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
output_notebook()
file = r'../test/cal_const_cd/test_rmse.csv'
calibration = pd.read_csv(file)
plot_source = ColumnDataSource(calibration)
p = figure()
p.circle(x='cd0', y='rmse',
        source=plot_source,
        size=10,color='green')
p.title.text='Green River Calibraion'
p.xaxis.axis_label='Drag Coefficient'
p.yaxis.axis_label='Root Mean Squared Error'
hover = HoverTool()
hover.tooltips=[
    ('Cd', '@cd0'),
    ('RMSE', '@rmse'),
    ('Discharge', '@Discharge')
]

p.add_tools(hover)

show(p)


Loading BokehJS ...

Or display results in a table

In [5]:
calibration

,Unnamed: 0,index,cd0,Discharge,rmse
0,0,0.0,0.00500,241.0,0.121602
1,1,1.0,0.00525,241.0,0.108868
2,2,2.0,0.00550,241.0,0.097340
3,3,3.0,0.00575,241.0,0.085476
4,4,4.0,0.00600,241.0,0.075601
5,5,5.0,0.00625,241.0,0.067969
6,6,6.0,0.00650,241.0,0.062545
7,7,7.0,0.00675,241.0,0.057815
8,8,8.0,0.00700,241.0,0.055246
9,9,9.0,0.00725,241.0,0.055306


Example 2: Calibrating to a vairabl Cd. The iRIC FaSTMECH (FM) project is located in the folder test/test_var_cd. Note that the Geographic Data | roughness has 2 converage polygons with values 0 and 1. 

In your own project make sure that these values have been mapped to the grid (ie. From the menu Grid->Attributes Mapping->Execute. Also that the Calculations Conditions have been specified and saved. The following example illustrates how to vary the value of roughness over a range specified in the .ini file and determine a calibrated roughness for the reach.

Note: For your own project you should spend some time calibrating the project in the iRIC application before launching this code to know that you have a good set of calculation conditions, boundary conditions, and some bounds as to what the roughness is.

In [7]:
# magic command to capture output of model output in calibration loop.  
#  Output is displayed in next cell command
# comment line below if you want to see in real-time.  
%%capture cal2_output

from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import plotly.plotly as py
import plotly.graph_objs as go
import time

cal2 = fm_cal('../test/test_varcd_config.ini')
cal2.initialize()

fig = plt.figure()
ax = fig.add_subplot(111)
cax = make_axes_locatable(ax).append_axes('right', size='5%', pad='2%')
plt.ion()

fig.show()
fig.canvas.draw()
x = np.arange(cal2.mcdmin['0'], cal2.mcdmax['0'], cal2.mcdinc['0'])
numx = len(x)
y = np.arange(cal2.mcdmin['1'], cal2.mcdmax['1'], cal2.mcdinc['1'])
numy = len(y)
X, Y = np.meshgrid(x, y, indexing='ij')
tcd ={}
tcdind = {}
tcount = 0
tmpz = np.zeros(shape=(numx,numy))
for cdind0, cd0 in enumerate(x):
    tcd[0] = cd0
    tcdind[0] = cdind0
    for cdind1, cd1 in enumerate(y):
        tcd[1] = cd1
        tcdind[1] = cdind1
        numind = len(tcdind)
        # simres = cal.update_var(cdind0, cd0, cdind1, cd1)
        simres = cal2.update_var(tcount, tcd)
        tmpz[cdind0,cdind1] = simres.loc[tcount, 'rmse']
        # tmpz =np.reshape(simres['rmse'].values,(numx,numy))
        cs = ax.contourf(X, Y, tmpz, 20, cmap=plt.cm.bone_r)
        ax.set_xlabel('cd0')
        ax.set_ylabel('cd1')
        cbar = fig.colorbar(cs,cax=cax)
        cbar.ax.set_ylabel('RMSE')
        #     ax.plot(simres[:cdind,0], simres[:cdind,1], '-o')  # fit the line
        fig.canvas.update()
        fig.canvas.flush_events()
        plt.pause(1)
        plt.draw()
        tcount+=1
#     fig.canvas.draw()   # draw
#     time.sleep(0.5)    #sleep
plt.savefig('testfig_varcd.png', dpi=300)
cal.resdf.to_csv(cal.rmse_file)
tmp = 0

UsageError: Line magic function `%%capture` not found.


In [ ]:
cal = fm_cal('../test/test_varcd_config.ini')
cal.initialize()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()
x = np.arange(cal.cd0min, cal.cd0max, cal.cd0inc)
y = np.arange(cal.cd1min, cal.cd1max, cal.cd1inc)
X,Y = np.meshgrid(x,y)
for cdind0, cd0 in enumerate(np.arange(cal.cd0min, cal.cd0max, cal.cd0inc)):
    for cdind1, cd1 in enumerate(np.arange(cal.cd1min, cal.cd1max, cal.cd1inc)):
        simres = cal.update_var(cdind0, cd0, cdind1, cd1)
        ax.contourf(X, Y, simres, 10, cmap=plt.cm.bone)
#     ax.plot(simres[:cdind,0], simres[:cdind,1], '-o')  # fit the line
        plt.pause(1)
        plt.draw()
#     fig.canvas.draw()   # draw
#     time.sleep(0.5)    #sleep
    
tmp = 0